# Profiling on a fast disk

(This is why we're using an AWS instance; it's an `i2.2xlarge` with a large, fast SSD.)

In [1]:
import time
import glob

import numpy
import ROOT

Welcome to JupyROOT 6.11/01


Start with some large files.

In [2]:
# filelist = sorted(glob.glob("/mnt/data/SingleElectronRun2015D_16Dec2015_v1/SingleElectron/CRAB3/160418_185159/000*/*.root"))
filelist = sorted(glob.glob("/mnt/data/SingleElectron_uncompressed/000*/*.root"))
fname = filelist[0]

In [3]:
f = ROOT.TFile(fname)
t = f.Get("Events")

Warning in <TClass::Init>: no dictionary for class baconhep::TEventInfo is available
Warning in <TClass::Init>: no dictionary for class baconhep::TElectron is available


Take all 4-byte branches of electrons (data were triggered on electrons).

In [4]:
branches = ["Electron_"]
for base in "Electron", "Info":    
    for b in t.GetBranch(base).GetListOfBranches():
        try:
            if t.GetNumpyIteratorInfo(b)[0][1].itemsize == 4:   # if the type has 4 bytes
                branches.append(b.GetName())                    # put it in the list
        except ValueError:                                      # unless it's an unhandled type
            pass                                                # (such as std::bitset)

In [5]:
len(branches)

88

In [6]:
set(x[1] for x in t.GetNumpyIteratorInfo(*branches))

{dtype('uint32'), dtype('int32'), dtype('float32')}

Maybe you want to (X) out of presentation mode to see the output better.

In [ ]:
totalEvents = 0
totalBytes = 0
startTime = time.time()

for findex, fname in enumerate(filelist):
    f = ROOT.TFile(fname)
    t = f.Get("Events")

    for stuff in t.GetNumpyIterator(*branches, return_new_buffers=False):
        start, end = stuff[:2]
        data = stuff[2:]
        totalEvents += end - start
        totalBytes += sum(4 * len(x) for x in data)        # all the branches are 4 bytes wide

    if findex % 10 == 0:                                   # print out every 10 files
        elapsedTime = time.time() - startTime
        ev_per_s = totalEvents / elapsedTime
        mb_per_s = totalBytes / elapsedTime / 1024**2
        print("events: {}\ttime: {}\trate: {} kHz, {} MB/s".format(
            totalEvents, elapsedTime, ev_per_s*1e-3, mb_per_s))

events: 24225	time: 0.230262041092	rate: 105.206224548 kHz, 35.3170309457 MB/s
events: 214194	time: 1.9312081337	rate: 110.91191895 kHz, 37.232394667 MB/s
events: 377177	time: 3.32416319847	rate: 113.465247486 kHz, 38.0895301011 MB/s
events: 556816	time: 4.62975716591	rate: 120.268942851 kHz, 40.3734854541 MB/s
events: 738987	time: 5.53619909286	rate: 133.482735647 kHz, 44.8092679478 MB/s
events: 930036	time: 6.39372205734	rate: 145.460811662 kHz, 48.8302285241 MB/s
events: 1100021	time: 7.47809815407	rate: 147.099032045 kHz, 49.380168228 MB/s
events: 1271044	time: 8.50694608688	rate: 149.412490337 kHz, 50.1567808137 MB/s
events: 1450551	time: 9.48562002182	rate: 152.921052779 kHz, 51.3345819266 MB/s
events: 1631415	time: 10.1970560551	rate: 159.988823361 kHz, 53.7071855766 MB/s
events: 1809581	time: 10.7910871506	rate: 167.692186593 kHz, 56.2931534585 MB/s
events: 1971691	time: 11.3749570847	rate: 173.336126486 kHz, 58.1877866012 MB/s
events: 2178278	time: 11.9752480984	rate: 181.8983

   * If we do this over recently accessed data, we see the effect of Linux file-page caching (scaling up to 100 MB/s).
   * If we let it run for a while, the starting rate of 45 MB/s sinks to 15 MB/s or less.
   * This slow-down is not "cured" unless we re-start the process (Python+ROOT). However, I don't see any evidence of a memory leak in `htop`.
   * Moreover, the disk's raw sequential rate is 380 MB/s.

In [14]:
!time cat /mnt/data/DYJetsToLL_M_50_HT_200to400_13TeV_2/* > /dev/null

0.00user 1.14system 0:09.44elapsed 12%CPU (0avgtext+0avgdata 1860maxresident)k
7265312inputs+0outputs (0major+138minor)pagefaults 0swaps


In [16]:
!du -ms /mnt/data/DYJetsToLL_M_50_HT_200to400_13TeV_2

3548	/mnt/data/DYJetsToLL_M_50_HT_200to400_13TeV_2


In [17]:
3548 / (9.44)

375.8474576271187